# Environment Basics

In [1]:
!pip install gymnasium==0.27.0
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.1/879.1 kB 11.1 MB/s eta 0:00:0000:0100:01


In [4]:
import gymnasium as gym
import seaborn as sns
from tqdm import tqdm

In [5]:
env = gym.make("Blackjack-v1", sab=True, render_mode="rgb_array")

### Observing the environment

In [6]:
# Resets the environment to get the first observation
done = False
observation, info = env.reset()

#observation = (16, 9, False)


#observation consistes of tuple with 3 values
#1. The players current sum
#2. Values of the dealers face-up card
#3. Boolean whether the player holds a usable ace (it is usuable if it counts as 11 without busting)

In [ ]:
#sample a random action  
action = env.action_space.sample()

#execute the actions in our environment and receiece info after taking the steo
observation, reward, terminated,truncated, info = env.step(action)

#observation=(24,10,False)
#reward=-1.0
#terminated=True
#truncated=False
#info={}
